In [1]:
import os

In [13]:
class ControllerGenerator:
  def __init__(self, endpoint_namespace, endpoint_route, route_singular, route_actions, module_name, object_attributes, object_params={}, filter_params="", sort_params=""):
    self.endpoint_namespace = endpoint_namespace
    self.endpoint_route = endpoint_route
    self.route_actions = route_actions
    self.module_name = module_name
    self.object_attributes = object_attributes
    self.object_params = object_params
    self.filter_params = filter_params
    self.sort_params = sort_params
    self.route_singular = route_singular
    
    self.directory = f'app/controllers/{endpoint_namespace}'
    self.filename = f"{endpoint_route}_controller.rb"

    os.makedirs(self.directory, exist_ok=True)
    self.filepath = os.path.join(self.directory, self.filename)

    self.serializer_name = f"{module_name}::{route_singular.capitalize()}Serializer"
    self.route_class_name = f"{module_name}::{endpoint_route.capitalize()}"
    self.controller_name = f"{self.route_class_name}Controller"

  def index_function_lines(self):
    return [
      "def index",
      f'''use_case = {self.route_class_name}::IndexCase.new(
        filter_parameters: {self.filter_params},
        sort_params: {self.sort_params},
        page_parameters: page_parameters)\n''',
        "result = use_case.perform\n",
        f'''render json: result.items,
                      meta: result.pagination,
                      meta_key: :pagination,
                      each_serializer: {self.serializer_name}''',
        "end\n"
    ]
  
  def show_function_lines(self):
    return [
      "def show",
      f"use_case = {self.route_class_name}::ShowCase.new(account: @account, id: @{self.route_singular}.id)\n",
      "result = use_case.perform\n",
      f"render json: result, serializer: {self.serializer_name}",
      "end\n"
    ]
  
  def create_function_lines(self):
    return [
      "def create",
      f"use_case = {self.route_class_name}::CreateCase.new({self.route_singular}_params.merge(account: @account))\n",
      "result = use_case.perform\n",
      "if result.errors.any?",
      "render json: { errors: result.errors }, status: :unprocessable_entity",
      "else",
      f"render json: result.record, serializer: {self.serializer_name}, status: :created",
      "end",
      "end\n"
    ]
  
  def update_function_lines(self):
    return [
      "def update",
      f"use_case = {self.route_class_name}::UpdateCase.new({self.route_singular}_params.merge(id: @{self.route_singular}.id, account: @account))\n",
      "result = use_case.perform\n",
      "if result.errors.any?",
      "render json: { errors: result.errors }, status: :unprocessable_entity",
      "else",
      f"render json: result.record, serializer: {self.serializer_name}, status: :ok",
      "end",
      "end\n"
    ]
  
  def destroy_function_lines(self):
    return [
      "def destroy",
      f"use_case = {self.route_class_name}::DestroyCase.new(id: @{self.route_singular}.id, account: @account)\n",
      "use_case.perform\n",
      "if use_case.errors.any?",
      "render json: { errors: use_case.errors }, status: :unprocessable_entity",
      "else",
      "head :no_content",
      "end",
      "end\n"
    ]
  
  def before_actions(self):
    actions = []

    if "R" in self.route_actions:
      actions.append(":show")
    if "U" in self.route_actions:
      actions.append(":update")
    if "D" in self.route_actions:
      actions.append(":destroy")

    return actions
      
  def get_lines(self):
    lines = [
      "# frozen_string_literal: true",
      "\n",
      f"class {self.controller_name} < ApplicationController",
    ]

    before_actions = self.before_actions()
    if len(before_actions) > 0:
      lines.append(f"before_action :set_{self.route_singular}, only: [{', '.join(before_actions)}]\n")

    if "R" in self.route_actions:
      for line in self.index_function_lines():
        lines.append(line)

    if "C" in self.route_actions:
      for line in self.create_function_lines():
        lines.append(line)

    if "R" in self.route_actions: # preserving order of index, create, show, so checking for 'R' twice
      for line in self.show_function_lines():
        lines.append(line)

    if "U" in self.route_actions:
      for line in self.update_function_lines():
        lines.append(line)

    if "D" in self.route_actions:
      for line in self.destroy_function_lines():
        lines.append(line)

    lines.append("private\n")

    if "C" in self.route_actions or "U" in self.route_actions:      
      if len(self.object_attributes) > 0:
        # object_param_items = ''
        
        object_param_items = ', '.join(':' + el for el in self.object_attributes)
        # for idx, item in enumerate(self.object_attributes):
        #   object_param_items += f':{item}'

        #   # if idx != len(self.object_attributes) - 1:
        #   #   object_param_items += ', '

        object_param_lines = [
          f"def {self.route_singular}_params",
          f'''params.require(:{list(self.object_params.keys())[0]})
                    .permit({object_param_items})''',
          "end\n"
        ]
          
        for line in object_param_lines:
          lines.append(line)

    filter_param_lines = [
      "def filter_params",
      f"params.permit({self.filter_params})",
      "end\n"
    ]

    if len(self.filter_params) > 0:
      for line in filter_param_lines:
        lines.append(line)

    sort_param_lines = [
      "def sort_parameters",
      f"params.permit(sort: [{self.sort_params}])",
      "end\n"
    ]

    if len(self.sort_params) > 0:
      for line in sort_param_lines:
        lines.append(line)

    set_route_singular_lines = [
      f"def set_{self.route_singular}",
      f"@{self.route_singular} = @account.{self.endpoint_route}.find(params[:id])",
      "end"
    ]

    for line in set_route_singular_lines:
        lines.append(line)

    lines.append("end")

    return lines
  
  def write_to_file(self):
    with open(self.filepath, 'w') as file:
      for line in self.get_lines():
        file.write(f"{line}\n")